In [1]:
#Run this if you want to force to use CPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [10]:
#!/usr/bin/env python3
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from qkeras import *
from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm

from qkeras.utils import _add_supported_quantized_objects
co = {}
_add_supported_quantized_objects(co)


# === RUTA LOCAL (ajustada a tu PC) ===
MODEL_DIR = "models/CNN/"


model_files = {
    "student": os.path.join(MODEL_DIR, "studentModelMnistCNN.h5"),
    "teacher": os.path.join(MODEL_DIR, "teacherModelMnistCNN.h5"),
    "teacher_qat": os.path.join(MODEL_DIR, "teacherModelQATMnistCNN.h5")
}

# === Carga MNIST ===
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_test = x_test.astype("float32") / 255.0
x_test_flat = x_test.reshape((x_test.shape[0], -1))

results = []

def safe_load_model(path):
    try:
        model = load_model(path,custom_objects=co)
        return model, None
    except Exception as e:
        return None, str(e)

for name, path in model_files.items():
    info = {"model": name, "path": path, "exists": os.path.exists(path)}

    if not os.path.exists(path):
        info["error"] = "file not found"
        results.append(info)
        continue

    info["filesize_bytes"] = os.path.getsize(path)

    model, err = safe_load_model(path)
    if err:
        info["error"] = f"load_error: {err}"
        results.append(info)
        continue

    info["loaded"] = True

    # Parámetros
    info["params_count"] = model.count_params()

    # Sparsity
    weights = model.get_weights()
    total = sum(w.size for w in weights)
    zeros = sum(np.sum(w == 0.0) for w in weights)

    info["weights_total"] = total
    info["weights_zeros"] = zeros
    info["sparsity_fraction"] = zeros / total

    # Evaluación
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    loss, acc = model.evaluate(x_test_flat, y_test, verbose=0)
    info["eval_loss"] = loss
    info["eval_accuracy"] = acc

    # Latencia
    model.predict(x_test_flat[:128], batch_size=128, verbose=0)
    t0 = time.time()
    model.predict(x_test_flat, batch_size=256, verbose=0)
    t1 = time.time()
    info["inference_time_total_s"] = t1 - t0
    info["inference_time_per_sample_ms"] = (t1 - t0) / len(x_test_flat) * 1000

    results.append(info)

# === DataFrame ===
df = pd.DataFrame(results)
print("\n=== Resumen ===")
print(df.to_string(index=False))

# === Gráficas ===
if "eval_accuracy" in df:
    df.set_index("model")["eval_accuracy"].plot(kind="bar",
                                                title="Accuracy MNIST")
    plt.ylabel("Accuracy")
    plt.show()

if "sparsity_fraction" in df:
    df.set_index("model")["sparsity_fraction"].plot(kind="bar",
                                                    title="Sparsity")
    plt.ylabel("Zeros Fraction")
    plt.show()

# === CSV ===
df.to_csv("mnist_summary.csv", index=False)
print("\nResumen guardado en mnist_summary.csv")


/home/pinpa/anaconda3/envs/mf_env/lib/python3.11/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer LecunUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


ValueError: in user code:

    File "/home/pinpa/anaconda3/envs/mf_env/lib/python3.11/site-packages/keras/engine/training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "/home/pinpa/anaconda3/envs/mf_env/lib/python3.11/site-packages/keras/engine/training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/pinpa/anaconda3/envs/mf_env/lib/python3.11/site-packages/keras/engine/training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "/home/pinpa/anaconda3/envs/mf_env/lib/python3.11/site-packages/keras/engine/training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "/home/pinpa/anaconda3/envs/mf_env/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/pinpa/anaconda3/envs/mf_env/lib/python3.11/site-packages/ke

In [ ]:
#!/usr/bin/env python3
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist

# --- QKeras imports ---
from qkeras.utils import _add_supported_quantized_objects

# Registrar objetos cuantizados
co = {}
_add_supported_quantized_objects(co)


# === RUTA LOCAL (ajustada a tu PC) ===
MODEL_DIR = "models/CNN/"

model_files = {
    "student": os.path.join(MODEL_DIR, "studentModelMnistCNN.h5"),
    "teacher": os.path.join(MODEL_DIR, "teacherModelMnistCNN.h5"),
    "teacher_qat": os.path.join(MODEL_DIR, "teacherModelQATMnistCNN.h5")
}


# === Carga MNIST ===
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalización
x_test = x_test.astype("float32") / 255.0

# Añadir canal: (N, 28, 28, 1)
x_test = x_test[..., tf.newaxis]


# === Función de carga segura ===
def safe_load_model(path):
    try:
        model = load_model(path, custom_objects=co)
        return model, None
    except Exception as e:
        return None, str(e)


results = []

for name, path in model_files.items():

    info = {"model": name, "path": path, "exists": os.path.exists(path)}

    # Verificar archivo
    if not os.path.exists(path):
        info["error"] = "file not found"
        results.append(info)
        continue

    info["filesize_bytes"] = os.path.getsize(path)

    model, err = safe_load_model(path)
    if err:
        info["error"] = f"load_error: {err}"
        results.append(info)
        continue

    info["loaded"] = True

    # Parámetros
    info["params_count"] = model.count_params()

    # Sparsity
    weights = model.get_weights()
    total = sum(w.size for w in weights)
    zeros = sum(np.sum(w == 0.0) for w in weights)

    info["weights_total"] = total
    info["weights_zeros"] = zeros
    info["sparsity_fraction"] = zeros / total

    # Evaluación
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    info["eval_loss"] = loss
    info["eval_accuracy"] = acc

    # === Latencia ===
    # Calentamiento
    model.predict(x_test[:256], batch_size=256, verbose=0)

    t0 = time.time()
    model.predict(x_test, batch_size=256, verbose=0)
    t1 = time.time()

    info["inference_time_total_s"] = t1 - t0
    info["inference_time_per_sample_ms"] = (t1 - t0) / len(x_test) * 1000

    results.append(info)


# === DataFrame ===
df = pd.DataFrame(results)
print("\n=== Resumen ===")
print(df.to_string(index=False))


# === Gráficas ===
plt.figure()
df.set_index("model")["eval_accuracy"].plot(kind="bar", title="Accuracy MNIST")
plt.ylabel("Accuracy")
plt.show()

plt.figure()
df.set_index("model")["sparsity_fraction"].plot(kind="bar", title="Sparsity")
plt.ylabel("Zeros Fraction")
plt.show()


# === CSV ===
df.to_csv("mnist_summary.csv", index=False)
print("\nResumen guardado en mnist_summary.csv")
